In [3]:
import cv2 
from PIL import Image 
import numpy as np 
import pandas as pd 
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split    

img_dir = 'train'
mask_dir = 'train/masks'
excel_file = 'train/classif.xlsx'
data_feature_file = 'data_features.csv'

# Load images
def load_images(img_dir, count):
    images = []
    for i in range(1, count + 1):
        img_path = os.path.join(img_dir, f"{i}.jpg")
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB format
            images.append(img)
        else:
            print(f"Image {img_path} not found.")
    return images

# Load masks
def load_masks(mask_dir, count):
    masks = []
    for i in range(1, count + 1):
        mask_path = os.path.join(mask_dir, f"binary_{i}.tif")
        if os.path.exists(mask_path):
            mask = Image.open(mask_path)
            mask = np.array(mask)
            # 检查掩码是否为灰度图像
            if mask.ndim == 3:
                mask = rgb2gray(mask)
            masks.append(mask)
            
        else:
            print(f"Mask {mask_path} not found.")
    return masks

# Load classification file
def load_classification(excel_file):
    if os.path.exists(excel_file):
        return pd.read_excel(excel_file)
    else:
        print(f"Excel file {excel_file} not found.")
        return None
    
# Load features
def load_features(data_feature_file):
    if os.path.exists(data_feature_file):
        return pd.read_csv(data_feature_file)
    else:
        print(f"Feature file {data_feature_file} not found.")
        return None    
    
images = load_images(img_dir, 250)
masks = load_masks(mask_dir, 250)
classif_df = load_classification(excel_file)   
features_df = load_features(data_feature_file)  

# 处理标签
def process_labels(df):
    bug_types = []
    species = []
    
    for _, row in df.iterrows():
        bugs = row['bug_type']
        species_info = row['species']
        
        # 处理x2
        if ' x2' in species_info:
            bugs = [bugs.replace(' x2','')] * 2
            species_info = [species_info.replace(' x2','')] * 2
        else:
            bugs = bugs.split(' & ')
            species_info = species_info.split(' & ')
        
        # 处理问号
        bugs = [bug.replace(' ?', '') for bug in bugs]
        species_info = [specie.replace(' ?', '') for specie in species_info]
        
        bug_types.append(bugs)
        species.append(species_info)
    
    df['bug_type'] = bug_types
    df['species'] = species
    return df

classif_df = process_labels(load_classification(excel_file))
# 使用MultiLabelBinarizer转换标签
mlb_bug_type = MultiLabelBinarizer()
mlb_species = MultiLabelBinarizer()
bug_type_encoded = mlb_bug_type.fit_transform(classif_df['bug_type'])
species_encoded = mlb_species.fit_transform(classif_df['species'])

# 转换为DataFrame
bug_type_df = pd.DataFrame(bug_type_encoded, columns=mlb_bug_type.classes_)
species_df = pd.DataFrame(species_encoded, columns=mlb_species.classes_)
#以及其他数据集
images = load_images(img_dir, 250)
masks = load_masks(mask_dir, 250)
features_df = load_features(data_feature_file)  



# 合并所有数据
all_data_df = pd.concat([classif_df.drop(columns=['bug_type', 'species']), bug_type_df, species_df, features_df], axis=1)


Mask 1 shape: (4000, 6000)
Mask 2 shape: (4000, 6000)
Mask 3 shape: (4000, 6000)
Mask 4 shape: (4000, 6000)
Mask 5 shape: (4000, 6000)
Mask 6 shape: (4000, 6000)
Mask 7 shape: (4000, 6000)
Mask 8 shape: (4000, 6000)
Mask 9 shape: (4000, 6000)
Mask 10 shape: (4000, 6000)
Mask 11 shape: (4000, 6000)
Mask 12 shape: (4000, 6000)
Mask 13 shape: (4000, 6000)
Mask 14 shape: (4000, 6000)
Mask 15 shape: (4000, 6000)
Mask 16 shape: (6000, 4000)
Mask 17 shape: (4000, 6000)
Mask 18 shape: (4000, 6000)
Mask 19 shape: (4000, 6000)
Mask 20 shape: (4000, 6000)
Mask 21 shape: (4000, 6000)
Mask 22 shape: (4000, 6000)
Mask 23 shape: (4000, 6000)
Mask 24 shape: (4000, 6000)
Mask 25 shape: (4000, 6000)
Mask 26 shape: (4000, 6000)
Mask 27 shape: (4000, 6000)
Mask 28 shape: (4000, 6000)
Mask 29 shape: (4000, 6000)
Mask 30 shape: (4000, 6000)
Mask 31 shape: (4000, 6000)
Mask 32 shape: (4000, 6000)
Mask 33 shape: (4000, 6000)
Mask 34 shape: (4000, 6000)
Mask 35 shape: (4000, 6000)
Mask 36 shape: (4000, 6000)
M

MemoryError: Unable to allocate 22.9 MiB for an array with shape (4000, 6000) and data type uint8

随机森林和支持向量机部分

In [4]:
    # Prepare feature matrix and target vector
X = all_data_df.drop(columns=['ID']).values
y = bug_type_encoded[:,0]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(200, 49)
(200,)


In [5]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(RandomForestClassifier(n_estimators=100), X_train, y_train, cv=5)
scores

array([1.   , 1.   , 0.975, 1.   , 1.   ])

In [6]:

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=3)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
print(f"Random Forest Accuracy: {rf_accuracy}")

# SVM
svm_model = SVC(kernel='linear', probability=True, C=0.1, random_state=42)
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_pred)
print(f"SVM Accuracy: {svm_accuracy}")

# Evaluate models on training set
rf_train_pred = rf_model.predict(X_train)
rf_train_accuracy = accuracy_score(y_train, rf_train_pred)
print(f"Random Forest Training Accuracy: {rf_train_accuracy}")

svm_train_pred = svm_model.predict(X_train)
svm_train_accuracy = accuracy_score(y_train, svm_train_pred)
print(f"SVM Training Accuracy: {svm_train_accuracy}")


Random Forest Accuracy: 1.0
SVM Accuracy: 1.0
Random Forest Training Accuracy: 1.0
SVM Training Accuracy: 1.0


随机森林和支持向量机2.0

In [13]:
# 合并特征和标签数据
labels_df = bug_type_df  # 或者使用 species_df 作为标签

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(features_df, labels_df, test_size=0.2, random_state=42)

# 确保数据集大小一致
print(f"训练集大小: {X_train.shape[0]}, 测试集大小: {X_test.shape[0]}")

# 训练随机森林分类器
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# 预测并计算准确度
y_pred_rf = rf_classifier.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"随机森林分类准确度: {accuracy_rf}")


# 使用OneVsRest策略做svm
svm_classifier = OneVsRestClassifier(SVC(kernel='linear', random_state=42))
svm_classifier.fit(X_train, y_train)

# 预测并计算准确度
y_pred_svm = svm_classifier.predict(X_test)

# 计算准确度
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"支持向量机分类准确度 (One-vs-Rest): {accuracy_svm}")



训练集大小: 200, 测试集大小: 50
随机森林分类准确度: 0.5
支持向量机分类准确度 (One-vs-Rest): 0.42


In [2]:
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import os

def extract_sift_features(images, masks):
    sift = cv2.SIFT_create()
    all_features = []

    for img, mask in zip(images, masks):
        keypoints, descriptors = sift.detectAndCompute(img, mask)
        if descriptors is not None:
            all_features.append(descriptors)
        else:
            print("No descriptors found for an image.")
    
    return all_features

sift_features = extract_sift_features(images, masks)

from sklearn.cluster import KMeans

# 聚类数目（可调）
n_clusters = 100

# 将所有特征拼接成一个矩阵
all_descriptors = np.vstack(sift_features)

# 使用KMeans进行聚类
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(all_descriptors)

# 创建Bag of Words表示
def create_bow_histogram(sift_features, kmeans):
    histograms = []
    for descriptors in sift_features:
        histogram = np.zeros(kmeans.n_clusters)
        if descriptors is not None:
            cluster_assignments = kmeans.predict(descriptors)
            for assignment in cluster_assignments:
                histogram[assignment] += 1
        histograms.append(histogram)
    return histograms

bow_histograms = create_bow_histogram(sift_features, kmeans)
bow_histograms = np.array(bow_histograms)

# 使用之前的标签数据
labels_df = bug_type_df  # 或者使用 species_df 作为标签

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(bow_histograms, labels_df, test_size=0.2, random_state=42)

# 训练随机森林分类器
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# 预测并计算准确度
y_pred_rf = rf_classifier.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"随机森林分类准确度: {accuracy_rf}")

# 数据标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 训练支持向量机分类器
svm_classifier = OneVsRestClassifier(SVC(kernel='linear', random_state=42))
svm_classifier.fit(X_train_scaled, y_train)

# 预测并计算准确度
y_pred_svm = svm_classifier.predict(X_test_scaled)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"支持向量机分类准确度 (One-vs-Rest): {accuracy_svm}")



error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\features2d\src\sift.dispatch.cpp:515: error: (-5:Bad argument) mask has incorrect type (!=CV_8UC1) in function 'cv::SIFT_Impl::detectAndCompute'


In [7]:
# CNN Model using PyTorch
class CNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Change to long for classification
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)  # Change to long for classification

# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Train CNN model
input_size = X_train.shape[1]
num_classes = len(np.unique(y_train))
cnn_model = CNN(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

for epoch in range(10):
    cnn_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = cnn_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Evaluate CNN model
cnn_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = cnn_model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

cnn_accuracy = correct / total
print(f"CNN Accuracy: {cnn_accuracy}")

# Autoencoder using PyTorch
class Autoencoder(nn.Module):
    def __init__(self, input_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Train autoencoder
autoencoder = Autoencoder(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

for epoch in range(10):
    autoencoder.train()
    running_loss = 0.0
    for inputs, _ in train_loader:
        optimizer.zero_grad()
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Extract features using autoencoder
autoencoder.eval()
encoded_train = autoencoder.encoder(X_train_tensor).detach().numpy()
encoded_test = autoencoder.encoder(X_test_tensor).detach().numpy()

# Train classifier on encoded features
encoded_rf_model = RandomForestClassifier(n_estimators=100)
encoded_rf_model.fit(encoded_train, y_train)
encoded_rf_pred = encoded_rf_model.predict(encoded_test)
encoded_rf_accuracy = accuracy_score(y_test, encoded_rf_pred)
print(f"Autoencoder + Random Forest Accuracy: {encoded_rf_accuracy}")

Epoch 1, Loss: 11.334738084248134
Epoch 2, Loss: 8.837949667658124
Epoch 3, Loss: 4.870382274900164
Epoch 4, Loss: 2.227934922490801
Epoch 5, Loss: 1.6491130760737829
Epoch 6, Loss: 0.9409172364643642
Epoch 7, Loss: 0.6558540889195034
Epoch 8, Loss: 0.5988060491425651
Epoch 9, Loss: 0.5732268861361912
Epoch 10, Loss: 0.5581392475536892
CNN Accuracy: 0.66
Epoch 1, Loss: 4874.539620535715
Epoch 2, Loss: 4863.079380580357
Epoch 3, Loss: 4823.917271205357
Epoch 4, Loss: 4849.474051339285
Epoch 5, Loss: 4854.509137834822
Epoch 6, Loss: 4858.455496651785
Epoch 7, Loss: 4855.201032366072
Epoch 8, Loss: 4904.05810546875
Epoch 9, Loss: 4881.471470424107
Epoch 10, Loss: 4898.097237723215
Autoencoder + Random Forest Accuracy: 0.52


In [8]:
class RealCNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RealCNN, self).__init__()
        self.conv1 = nn.Conv2d(input_size, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn1 = nn.BatchNorm2d(512)
        self.conv2 = nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn2 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(256, num_classes)
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.bn1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.bn2(x))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x
RealCNN_model = RealCNN(input_size, num_classes)
RealCNN_model.eval()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(RealCNN_model.parameters(), lr=0.001)
def train():
    with torch.no_grad():
        for epoch in range(10):
            RealCNN_model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = RealCNN_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                print(running_loss)

    

In [9]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

# 定义数据集类
class BeeDataset(Dataset):
    def __init__(self, img_dir, mask_dir, labels, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.labels = labels
        self.transform = transform
        self.image_ids = list(labels.index)

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_path = os.path.join(self.img_dir, f"{img_id}.jpg")
        mask_path = os.path.join(self.mask_dir, f"binary_{img_id}.tif")

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        label = self.labels.iloc[idx]
        return image, label

# 定义数据增强和预处理
data_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# 加载标签数据
labels_df = pd.read_excel('train/classif.xlsx')
labels_df['ID'] = labels_df.index + 1

# 创建数据集和数据加载器
train_dataset = BeeDataset(img_dir='train', mask_dir='train/masks', labels=labels_df, transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

num_classes = len(labels_df['bug_type'].unique())
model = SimpleCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# 评估模型
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in train_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"CNN Accuracy: {accuracy}")


TypeError: __call__() got an unexpected keyword argument 'image'

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
import torch.nn.functional as F

class AugmentedDataset(Dataset):
    def __init__(self, tensor_data, tensor_labels, transform=None):
        self.data = tensor_data
        self.labels = tensor_labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label

# Define custom data augmentation transformations
def add_noise(data, noise_factor=0.1):
    noise = torch.randn_like(data) * noise_factor
    return data + noise

def scale_data(data, scale_factor=0.1):
    scale = torch.randn_like(data) * scale_factor + 1.0
    return data * scale

def custom_transform(data):
    data = add_noise(data)
    data = scale_data(data)
    return data

# Create augmented dataset with custom transformations
augmented_train_dataset = AugmentedDataset(X_train_tensor, y_train_tensor, transform=custom_transform)

# Create data loaders
train_loader = DataLoader(augmented_train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the CNN Model using PyTorch
class CNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Change to long for classification
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)  # Change to long for classification

# Initialize the CNN model
input_size = X_train.shape[1]
num_classes = len(np.unique(y_train))
cnn_model = CNN(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# Train CNN model with augmented data
for epoch in range(10):
    cnn_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = cnn_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Evaluate CNN model
cnn_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = cnn_model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

cnn_accuracy = correct / total
print(f"CNN Accuracy after data augmentation: {cnn_accuracy}")
